# Forward and Backward Process

In this notebook, we explore what the decoder is capable of doing with:  
1. A single image that has been noised at a specific step in the diffusion process
2. A full sequence of progressively noised images across all diffusion steps

## Setup

We begin by importing the necessary libraries and loading the pretrained diffusion model and scheduler.

What you can change in the `src/config.yaml`:
- You can select the model (and dataset) used by editing the `model` attribute
- You can select the number of generation time steps by editing the `model.timesteps` attribute
- You can select the specific step in the diffusion process by editing the `forward.timestep` attribute

In [ ]:
import functions
import diffusers
import torch
from tqdm import tqdm
from PIL import Image

In [ ]:
# setup
model_id = functions.config("model")
model = diffusers.UNet2DModel.from_pretrained(model_id)
scheduler = diffusers.DDIMScheduler.from_pretrained(model_id)
scheduler.set_timesteps(functions.config("model.timesteps"))

## Denoising a noised image

We start by loading an image and converting it into a tensor.  
Then we simulate the forward prcess by adding noise to it at a specific timestep (forward_timestep_index).  
Finally, we apply the decoder (reverse process) to try to reconstruct the original image from the noised version.  

In [ ]:
# loading and transforming an image
original_image = Image.open("../images/ddpm_9.png")
original_image = functions.image_as_tensor(original_image)
image_size = model.config.sample_size # get image size

We add noise to the image at the choses timestep in the diffusion process. Feel free to change the specific timestep by editing the `forward.timestep` attribute in the `src/config.yaml` file.

In [ ]:
# forward process
noised_image = scheduler.add_noise(original_image, functions.generate_noise(image_size), scheduler.timesteps[functions.config("forward.timestep")])
functions.tensor_as_image(noised_image)

We iteratively apply the decoder to remove noise and reconstruct the image step by step:

In [ ]:
# backward process
current = noised_image
for t in tqdm(scheduler.timesteps[functions.config("forward.timestep"):]):
    with torch.no_grad():
        predicted_noise = model(current, t).sample
        current = scheduler.step(predicted_noise, t, current).prev_sample
        functions.show_images(current)

We then compare the original image, the reconstructed image, and their difference to evaluate how well the model recovers lost information.

In [ ]:
# compare images
functions.show_images(original_image, current, original_image - current)

## Noise Accumulation and Decoder Output at Each Step

The following code generates a large image that visualizes how information is gradually lost through the progressive addition of noise during the diffusion process. This cell may take a very long time to execute. If you do not want to execute the cell yourself, you can take a look at an example result in `images/process_ddpm_7.png`.

- The first column shows the original image as processed by the encoder  
- Each subsequent column displays the output of the decoder applied to increasingly noised versions of the image  
- Each row corresponds to a specific step in the noise schedule: the further down, the more noise has been added before decoding. 

This visualization illustrates how, step by step, information degrades during the forward diffusion process (vertical axis). And how the decoder, step by step, reduces the noise to generate an image (horizontal axis). It also illustrates how the decoder's ability to recover the original image decreases as the input becomes more corrupted.

In [ ]:
# setup output image
sampling_steps = len(scheduler.timesteps)
output = Image.new("RGB", (image_size * sampling_steps, image_size * sampling_steps), (255, 255, 255))
output.paste(functions.tensor_as_image(original_image), (0, 0))

# output generation
with tqdm(total=sum(range(1, sampling_steps + 1))) as tqdm_bar:
    # forward process loop (vertical axis)
    for i in reversed(range(sampling_steps)):
        # generate new noise and add it to the image
        sampled_noise = functions.generate_noise(image_size)
        current = scheduler.add_noise(original_image, sampled_noise, scheduler.timesteps[i])
        output.paste(functions.tensor_as_image(current), (0, image_size * (sampling_steps - i)))
        # reverse process loop (horizontal axis)
        for j, t in enumerate(scheduler.timesteps[i:]):
            # denoise the current image
            with torch.no_grad():
                predicted_noise = model(current, t).sample
                current = scheduler.step(predicted_noise, t, current).prev_sample
                output.paste(functions.tensor_as_image(current), (image_size * (j + 1), image_size * (sampling_steps - i)))
            # update process bar
            tqdm_bar.update(1)

# save output image
output.save("../output/process.png")